## <center> Plotting B-spline curves defined by  `BasicBSpline.jl` </center>

Docs: https://hyrodium.github.io/BasicBSpline.jl/dev/

In [1]:
using BasicBSpline, PlotlyJS

function bsplcurve(p::Int, knot:: KnotVector{Float64}, ctrl:: Vector{Vector{T}}, N::Int) where T
    """
    p: degree of B-spline basis functions
    knot: the sequence of m knots, m-p >p 
    ctrl: the vector of control points; length(ctrl)= n =  m-p-1
    N: the number of points to be computed on the B-spline curve
    returns N points on the B-spline curve defined by the knots and ctrl
    """
    m = length(knot)
    BS = BSplineSpace{p}(knot) # B-spline space
    n  = dim(BS)
    @assert length(ctrl)==n
    I = domain(BS)
    a, b =extrema(I)
    t = LinRange(a, b, N)
    bs_curve= Vector{Float64}[]
    for s in t
        basis= [bsplinebasis(BS, i, s) for i in 1:n]
        push!(bs_curve, sum(ctrl[i,]*basis[i] for i in 1:n))
    end
    return bs_curve
end;    

WebIO._IJuliaInit()

In [2]:
function figbspline(ctrl:: Vector{Vector{T}}, bs_pts::Vector{Vector{Float64}}; 
                    width=600, height=600) where T
    # function to display a B-spline curve as a PlotlyJS.jl plot
    fig = Plot(scatter(x=[v[1] for v in ctrl], y =[v[2] for v in ctrl], 
                       name="ctrl polygon", marker_size=8))
    addtraces!(fig, scatter(x= [v[1] for v in bs_pts], y=[v[2] for v in bs_pts],
                            name="b-spline", mode="lines", line_color="red"))
    relayout!(fig, width=width, height=height) 
    return fig
end;   

**Example of B-spline curve containing the first and last  control point**

In [3]:
p = 3 #degree of BS polynomials
#end knots must have multipliciti p+1 to ensure that the B-spline curve contains the end control points
knot = KnotVector(1, 1, 1, 1, 1.5, 2.3, 3., 3.5, 3.5, 3.5, 3.5) 
m = length(knot)
#Give m-p-1  control points
#println(m-p-1)
ctrl=[[0., 0.45], [-0.5, 0.75], [-1.,2.0], [0.5, 3.2], 
      [1.6, 1.0], [1.3, -0.3], [0.75, -1.0]];
bs_pts = bsplcurve(p, knot, ctrl, 200)   
fig=figbspline(ctrl, bs_pts; height=450)
display(fig)

data: [
  "scatter with fields marker, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y"
]

layout: "layout with fields height, margin, template, and width"

In [4]:
savefig(fig, "cubic-B-spline.png", width=600, height=450, scale=1)

"cubic-B-spline.png"

**Closed B-spline curve**

Define the control points and the sequence of knots such that to get a closed B-spline curve, following  *J.J. Risler - Mathematical Methods for CAD*, Cambridge Univ Press.

In [5]:
p = 3
ictrl =  [[0., 90.], [-40, 72], [-50, 0], [-25, -15], [10,-10], [32,20], [25, 60]] #initial control points;
#extend the ictrl to ensure a closed B-spline curve
ctrl = vcat(ictrl, [ictrl[j] for j=1:p]); #by JJ Risler
n = length(ctrl)
u = Float64.(collect(1:n+p+1)) #knots
period = u[n+1]-u[p+1]
u[1:p] = u[n-p+1:n] .- period
u[n+2:n+p+1] = u[p+2:2*p+1] .+ period
knot = KnotVector(u)
bs_pts = bsplcurve(p, knot, ctrl, 200) 
fig = figbspline(ctrl, bs_pts; height=450)
relayout!(fig, title_text="Closed B-spline curve", title_x=0.45)
display(fig)

data: [
  "scatter with fields marker, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y"
]

layout: "layout with fields height, margin, template, title, and width"

In [6]:
savefig(fig, "closed-B-spline.png", width=600, height=450, scale=1)

"closed-B-spline.png"